# 데이터 전처리
모델의 더 나은 성능을 위해 아래의 항목들을 확인
- 중복되는 data가 존재하는 지 확인
- train data와 test data의 특수 문자 확인
- train data와 test data에서 얼마나 다양한 언어들이 사용되었는지 확인

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re

from transformers import AutoTokenizer

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (3.0.4)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


In [14]:
train_data = pd.read_csv("../../data/train.csv")

In [81]:
train_data["특수문자"] = "None"

### 중복 데이터 확인

In [ ]:
duplicated_data = train_data[train_data.duplicated(['text','target','url','date'], keep=False)]
duplicated_data.sort_values(by=['text'])    # 모든 항목이 중복되는 데이터는 없음을 확인

In [ ]:
duplicated_data = train_data[train_data.duplicated(['text'], keep=False)]
duplicated_data.sort_values(by=['text'])    # text가 중복되는 데이터는 없음을 확인

결론 : 모든 항목이 중복되거나, text 항목만 중복되는 데이터는 없었다. 그래서 중복 데이터를 제거할 필요는 없어보인다.

### 특수문자 확인

In [96]:
special_list = []
pattern = r"[^A-Za-zㄱ-힣0-9\s]"

for idx, data in enumerate(train_data["text"]):
    result = re.findall(pattern, data)
    if len(result) > 0:
        special_list.append(result)
        train_data.iloc[idx, 5] = str(result)

In [ ]:
train_data[train_data["특수문자"] != "None"]

In [99]:
# "특수문자" 열에서 중복되는 값들은 삭제
special_list = [item for sublist in special_list for item in sublist]
set_data = set(special_list)
print(set_data)

{'④', '⑫', 'ｇ', '‘', '蘭', '₂', '⅓', '⑪', '↔', '＋', '％', '·', '→', '樂', '×', '↓', '%', '③', '○', '∼', '…', '⑥', '⑤', '⅔', '.', '女', '金', '↑', 'ｍ', '℃', '②', '①'}


### 특수문자 제거

In [100]:
for idx, data in enumerate(train_data["text"]):
    preprocessed_data = re.sub(pattern, ' ', data)
    train_data.iloc[idx, 1] = preprocessed_data

In [102]:
train_data[train_data["특수문자"] != "None"]

,ID,text,target,url,date,특수문자
3,ynat-v1_train_00003,신간 블록체인혁명 2030 남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49,['·']
6,ynat-v1_train_00006,콜마비앤에이치 작년 영업익 352억원 54 5,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.18. 오후 4:45,"['…', '.', '%', '↑']"
7,ynat-v1_train_00007,SOil 1조2천억원 경유 납사 항공유 공급계약,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.05. 오전 10:21,"['·', '·']"
8,ynat-v1_train_00008,카카오 삼성화재 디지털 손해보험사 설립 추진,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.23. 오전 10:49,['·']
12,ynat-v1_train_00012,美웨스트버지니아 경선 샌더스 박빙 우세 전체 판세와는 무관,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.11. 오전 9:48,['…']
...,...,...,...,...,...,...
6992,ynat-v1_train_06992,HSD엔진 3분기 영업손실 130억원 적자전환,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.12. 오후 4:37,['…']
6993,ynat-v1_train_06993,새해 첫 NSC 상임위 개최 北제안 후속조치 마련 착수,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.02. 오후 2:14,['…']
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발변수에 촉각 왜 지금,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24,['…']
6998,ynat-v1_train_06998,문 대통령 1987 관람 깜짝 방문에 객석 환호 박수종합,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.07. 오후 5:23,"['…', '·']"


### klue/bert-base tokenizer의 한자 토큰 처리

In [2]:
# tokenizer가 한자를 처리하는 경우
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [9]:
# train data 중 일부를 가져옴
text = "朴대통령 정진석 원내대표·김광림 정책위의장에 축하蘭"
output = tokenizer(text, add_special_tokens=True)
tokenized_data = output["input_ids"]

In [10]:
decode_data = tokenizer.decode(tokenized_data)
decode_data

'[CLS] 朴 대통령 정진석 원내대표 · 김광림 정책위의장에 축하 [UNK] [SEP]'

In [14]:
pattern = r"[一-龥]"
text = re.sub(pattern, '', text)
text

'대통령 정진석 원내대표·김광림 정책위의장에 축하蘭'

위의 특수문자 처리를 통해 제거되는 한자와 r"[一-龥]" 을 사용해 제거되는 한자가 존재

### 한자를 가진 text 수 및 한자 종류 확인

In [17]:
pattern = r"[一-龥]"
result_list = []
chinese_list = []
for idx, data in enumerate(train_data["text"]):
    result = re.findall(pattern, data)
    if len(result) > 0:
        result_list.append(train_data.iloc[idx, 1])
        chinese_list.append(result)

In [18]:
# train data에 포함된 한자들
chinese_char = [item for sublist in chinese_list for item in sublist]
set_data = set(chinese_char)
print(set_data)

{'富', '父', '風', '文', '獨', '印', '證', '硏', '靑', '野', '中', '外', '脫', '潘', '賞', '駐', '港', '車', '大', '檢', '前', '號', '强', '色', '寒', '親', '千', '人', '對', '與', '北', '日', '故', '社', '勝', '四', '韓', '族', '秋', '伊', '黃', '尹', '新', '軍', '弗', '美', '戰', '曺', '佛', '亞', '比', '展', '朴', '企', '法', '崔', '重', '黨', '禹', '體', '核', '無', '詩', '銀', '孫', '南', '英', '行', '協', '安', '反'}


In [21]:
# 한자를 가진 text 수 : 917
result_list

['美웨스트버지니아 경선 샌더스 박빙 우세…전체 판세와는 무관',
 '北 한미일 연합훈련 비난…지배주의적 야망의 발로',
 '與 대구·경북 공천면접심사…현역 vs 진박 격돌',
 '이란 호르무즈 부근서 美드론 격추…美 이유없는 공격종합3보',
 '美공백 신속 대체하는 러…더타임스 시리아 분할 감독할 것',
 '佛 노란조끼 집회 폭력 사태에 파리 경찰청장 경질',
 '外人 주식 매도에 원달러 환율 하락폭 축소…1086원대',
 '朴대통령 한미일 대북압박 연대강화…北도발시 더 강력제재',
 '이란 美에 수감자 교환 제안…美는 거부',
 '朴대통령 담화에 與 진심어린 사죄 vs 野 분노 해소에 미흡',
 '朴대통령 중소기업인과 오찬…규제개혁·중기 지원의지 강조',
 '북 수소탄실험 北매체 주민들 국력 과시에 환희 선전',
 '北 김정은 군사격 타격수단 더 생산…신무기 개발의지 피력',
 '北 고려항공 中 칭다오 운항…관광객 수송 목적',
 '구글·페이스북 美인터넷 광고 작년 증가분 99% 싹쓸이',
 '미생 日정부 주최 미디어예술상 우수상…구성 뛰어나',
 '인도 美 취업삐자 수수료 인상은 부당 WTO 제소',
 '日외무상 中에 견제구…핵보유국에 핵탄두수 공개 요구하겠다',
 '文 1948년 건국절 주장대한민국 정통성 부정하는 얼빠진 주장',
 '北 연일 최순실 사태 비난공세…비극적 말로 피할수 없다',
 '北 조선중앙TV 당대회 김정은 총화보고 녹화방송3보',
 'NHK 北동해안서도 탄도 미사일 발사 준비 움직임',
 '日정부 여론전…주일 외교관 대상 수출규제 설명회 열어',
 '獨내각 석탄화력 퇴출지역 지원법 승인…2038년까지 53조 지원',
 '반기문 北과 대화를 향한 길 다시 찾아야 할 것3보',
 '증시신상품 한투운용 美뉴욕 오피스 빌딩 투자 펀드',
 'SKT 스마트그리드·KT 정보보안…통신사들 脫통신',
 '北김기남 대북확성기 방송 비난…전쟁 접경에로 몰아간다',
 '與 野 안보불안 조장黨…선거에 이용 안돼',
 '알자지라 이란 폭발물수거 美영상 사건

### 한자 제거

In [127]:
# 한자 전처리 시, 할 수 있는 방법
# 1. 그냥 제거

for idx, data in enumerate(train_data["text"]):
    preprocessed_data = re.sub(pattern, '', data)
    train_data.iloc[idx, 1] = preprocessed_data

In [ ]:
train_data

In [14]:
# 2. 한자를 한국어로 번역 -> 라이브러리 탐색 후 적용

# hanja 라이브러리 : 한자를 음으로 번역해주는 라이브러리, 北 -> "북"
import hanja
hanja.translate('女이다.', 'substitution')

'여이다.'

In [8]:
for idx, data in enumerate(train_data["text"]):
    translated_data = hanja.translate(data, 'substitution')
    train_data.iloc[idx, 1] = translated_data

In [9]:
train_data[:20]

,ID,text,target,url,date
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00
3,ynat-v1_train_00003,신간 블록체인혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러 혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15
5,ynat-v1_train_00005,배구연맹 이영일구 순천 KOVO커 부녕 대행업체 입찰 공고,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.07.10 16:47
6,ynat-v1_train_00006,콜마비앤에이치 작년 영업익 352억원…54.5%↑,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.18. 오후 4:45
7,ynat-v1_train_00007,SOil 1조2천억원 경유·납사·항공유 공급계약,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.05. 오전 10:21
8,ynat-v1_train_00008,카카오·삼성화재 디지털 손해보험사 설립 추진,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.23. 오전 10:49
9,ynat-v1_train_00009,긴급비상대책위 참석하는 김병준과 나경원,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.08. 오후 6:32


결론 : 한자->한글로 번역하니 오히려 성능 하락